In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical
from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [2]:
data = pd.read_csv('data/fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
train = data[data['Usage'] == 'Training']
test = data[data['Usage'] != 'Training']

In [4]:
train.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
test.head()

,emotion,pixels,Usage
28709,0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...,PublicTest
28710,1,156 184 198 202 204 207 210 212 213 214 215 21...,PublicTest
28711,4,69 118 61 60 96 121 103 87 103 88 70 90 115 12...,PublicTest
28712,6,205 203 236 157 83 158 120 116 94 86 155 180 2...,PublicTest
28713,3,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...,PublicTest


In [6]:
def prepare_data(data) :
    y_data = keras.utils.to_categorical(data['emotion'])
    X_data = np.zeros(shape=(len(data), 48, 48, 1))

    for i, row in enumerate(data.index):
        pixels = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        pixels = np.reshape(pixels, (48, 48)) 
        X_data[i, :, :, 0] = pixels / 255

    return X_data, y_data

In [7]:
X_train, y_train = prepare_data(train)
X_test, y_test = prepare_data(test)

print(X_train)
print(y_train[0])
print(X_test)
print(y_test[0])

[[[[0.2745098 ]
   [0.31372549]
   [0.32156863]
   ...
   [0.20392157]
   [0.16862745]
   [0.16078431]]

  [[0.25490196]
   [0.23921569]
   [0.22745098]
   ...
   [0.21960784]
   [0.20392157]
   [0.17254902]]

  [[0.19607843]
   [0.16862745]
   [0.21176471]
   ...
   [0.19215686]
   [0.21960784]
   [0.18431373]]

  ...

  [[0.35686275]
   [0.25490196]
   [0.16470588]
   ...
   [0.28235294]
   [0.21960784]
   [0.16862745]]

  [[0.30196078]
   [0.32156863]
   [0.30980392]
   ...
   [0.41176471]
   [0.2745098 ]
   [0.18039216]]

  [[0.30196078]
   [0.28235294]
   [0.32941176]
   ...
   [0.41568627]
   [0.42745098]
   [0.32156863]]]


 [[[0.59215686]
   [0.58823529]
   [0.57647059]
   ...
   [0.50588235]
   [0.54901961]
   [0.47058824]]

  [[0.59215686]
   [0.58431373]
   [0.58431373]
   ...
   [0.47843137]
   [0.55294118]
   [0.5372549 ]]

  [[0.59215686]
   [0.59215686]
   [0.61176471]
   ...
   [0.42745098]
   [0.48235294]
   [0.57254902]]

  ...

  [[0.7372549 ]
   [0.7372549 ]
   [0.4

In [8]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(BatchNormalization())

cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(BatchNormalization())

cnn_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.5))
cnn_model.add(BatchNormalization())

cnn_model.add(Flatten())

cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))

cnn_model.add(Dense(7, activation='softmax'))

cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 22, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       1

In [9]:
opt = keras.optimizers.Adam(lr=0.001)
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

In [10]:
%%time 

h1 = cnn_model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1, 
                   validation_data =(X_test, y_test))

Epoch 1/50
449/449 [==============================] - 441s 981ms/step - loss: 2.0366 - accuracy: 0.1998 - val_loss: 1.8330 - val_accuracy: 0.2320
Epoch 2/50
449/449 [==============================] - 433s 964ms/step - loss: 1.8191 - accuracy: 0.2383 - val_loss: 1.8016 - val_accuracy: 0.2469
Epoch 3/50
449/449 [==============================] - 433s 963ms/step - loss: 1.7842 - accuracy: 0.2552 - val_loss: 1.7677 - val_accuracy: 0.2675
Epoch 4/50
449/449 [==============================] - 434s 967ms/step - loss: 1.6950 - accuracy: 0.3102 - val_loss: 1.6370 - val_accuracy: 0.3522
Epoch 5/50
449/449 [==============================] - 433s 965ms/step - loss: 1.5687 - accuracy: 0.3741 - val_loss: 1.5148 - val_accuracy: 0.4341
Epoch 6/50
449/449 [==============================] - 434s 966ms/step - loss: 1.4959 - accuracy: 0.4156 - val_loss: 1.4772 - val_accuracy: 0.4494
Epoch 7/50
449/449 [==============================] - 433s 964ms/step - loss: 1.4187 - accuracy: 0.4543 - val_loss: 1.3496 -

In [11]:
cnn_model_json = cnn_model.to_json()
with open("cnn_model3.json", "w") as json_file:
    json_file.write(cnn_model_json)
cnn_model.save_weights("cnn_model3.h5")

In [12]:

print(cnn_model.predict(X_test))

[[2.8272259e-01 4.3034248e-02 2.3943886e-01 ... 3.0733800e-01
  1.4165446e-02 9.9614091e-02]
 [2.1229848e-01 8.4461056e-02 2.9325670e-01 ... 2.5992993e-01
  2.7148355e-02 7.4510492e-02]
 [7.8519589e-01 2.6815627e-02 6.3302711e-02 ... 9.0427905e-02
  1.8299579e-04 3.4012523e-02]
 ...
 [1.6122918e-01 9.1589550e-03 1.1666897e-01 ... 1.7394981e-01
  2.5857707e-02 1.9119185e-01]
 [7.4275680e-02 4.8887817e-04 2.7614560e-02 ... 3.9339799e-02
  9.8874941e-03 6.5787412e-02]
 [1.3546652e-01 4.6968907e-02 3.4052688e-01 ... 4.0111631e-01
  1.2373685e-02 5.4276325e-02]]
